# Setup Enviornment

In [1]:
#!git clone https://github.com/AntonMu/TrainYourOwnYOLO.git

#!pip install -r TrainYourOwnYOLO/requirements.txt
#!pip3 uninstall -y enum34

In [5]:

# clear images in the TrainYourOwnYOLO directories
! rm  TrainYourOwnYOLO/Data/Source_Images/Training_Images/*.jpg
! rm  TrainYourOwnYOLO/Data/Source_Images/Test_Images/*.jpg
! rm TrainYourOwnYOLO/Data/Source_Images/Training_Images/vott-csv-export/*.jpg

# Define Directories

In [2]:
import os
from pathlib import Path

from shared import *


Path('raw_labels').mkdir(parents=True, exist_ok=True)

BASE = Path('TrainYourOwnYOLO')
RAW_LABELS = Path('raw_labels')
SOURCE_IMAGES = Path('Source_Images')
TRAINING_IMAGES = BASE / Path('Data') / SOURCE_IMAGES / Path('Training_Images')
TEST_IMAGES = BASE / Path('Data') / SOURCE_IMAGES / Path('Test_Images')

Path(TRAINING_IMAGES / 'vott-csv-export').mkdir(parents=True, exist_ok=True)
LAEBLS_DIR = TRAINING_IMAGES / 'vott-csv-export'
YOLO_LABLES = LAEBLS_DIR / 'data_train.txt'


# configure s3 resources
s3_client = boto3.client('s3', region_name='us-east-1')
s3_res = make_s3_resource()
bucket = create_s3_bucket_obj(s3_res)

# Download raw labels from S3

In [3]:
key = 'final_labels.json'
local_dest = RAW_LABELS / 'final_labels.json'
s3_client.download_file(bucket.name, key, str(local_dest))

# download images to machine
image_ids = list_image_ids(bucket)
for image_id in image_ids:
    key = f'data/{image_id}'
    if (TRAINING_IMAGES/image_id).is_file():
        continue
    s3_client.download_file(bucket.name, key, str(TRAINING_IMAGES/image_id))

# Download and process Yolo Labels

In [4]:
import pandas as pd


def write_yolo_labels_file(label_json_path: str, fpath: str):

    df = pd.read_json(label_json_path).T.reset_index()
    df = df.rename(columns={'index': 'path'})
    df['path'] =  str(TRAINING_IMAGES.absolute()) + '/' + df['path']
    df['label'] =  0
    df = df[['path', 'xmin', 'ymin', 'xmax', 'ymax', 'label']]
    df.xmin = df.xmin.astype(int)
    df.ymin = df.ymin.astype(int)
    df.xmax = df.xmax.astype(int)
    df.ymax = df.ymax.astype(int)

    lines = []
    for index, row in df.iterrows():
        path, xmin, ymin, xmax, ymax, label = row
        line = f'{path} {xmin},{ymin},{xmax},{ymax},{label}'
        lines.append(line)

    with open(fpath, 'w+') as output:
        for line in lines:
            output.write(line)
            output.write("\n")
            
            
# execute
write_yolo_labels_file(label_json_path=local_dest, fpath=YOLO_LABLES)

# Train Model

In [ ]:
!python TrainYourOwnYOLO/2_Training/Download_and_Convert_YOLO_weights.py

In [ ]:
!python TrainYourOwnYOLO/2_Training/Train_YOLO.py --epochs 15

# Inference

In [ ]:
!python TrainYourOwnYOLO/3_Inference/Detector.py --confidence 0.6